predict creates images from a saved generator


In [ ]:

import numpy as np
from numpy import save
import pandas as pd 

# Input data files are available in the "../input/" directory.
#import os
#print(os.listdir("inputs"))

In [ ]:
import os
import time
import tensorflow as tf
#import glob
#from glob import glob
import datetime
import random
from PIL import Image
#import matplotlib
#matplotlib.use('agg')
#import matplotlib.pyplot as plt

#import seaborn as sns
#%matplotlib inline

# to choose which GPU should be used
#import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""
#os.environ["HIP_VISIBLE_DEVICES"]="1"


# this section will allow code to be run on a RTX GPU
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [ ]:
import tensorflow.compat.v1 as keras
#from keras.layers import Input, Dense, Reshape, Flatten, Dropout, GaussianNoise
#from keras.layers import BatchNormalization, Activation, ZeroPadding2D
#from keras.layers.advanced_activations import LeakyReLU
#from keras.layers.convolutional import UpSampling2D, Conv2D , Conv2DTranspose
from tensorflow.compat.v1.models import Sequential, Model, load_model
from tensorflow.compat.v1.optimizers import Adam


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Hyperparameters
IMAGE_SIZE = 1024
NOISE_SIZE = 100

#runtime 
MODEL_EPOCH = 1712 # current rect art 
MODEL_EPOCH = 306
LOAD_WEIGHTS_EP = 19

SAMPLES_TO_SHOW = 10
EPOCHS_TO_RUN = 50

MODEL_NAME = 'models/' + 'gan-' + str(IMAGE_SIZE) + '-' + str(NOISE_SIZE) + '-model.' + str(MODEL_EPOCH) + '.' + 'h5'

In [ ]:

model = load_model(MODEL_NAME)
#model.summary()
model.load_weights("weights/gan-" + str(IMAGE_SIZE) + "-100-weights." + str(LOAD_WEIGHTS_EP) + ".h5", by_name=True)
generator = model.get_layer(name="model_2")
#generator = model


In [ ]:
def predict(input_z, epoch):
    samples = generator.predict(input_z[:SAMPLES_TO_SHOW])
    
    # added for comparing the value
    np.savetxt(noise_file_handle,input_z[:SAMPLES_TO_SHOW], delimiter=',')
    
    sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    name = OUTPUT_DIR + "samples"
    print("save images")
    for index in range(SAMPLES_TO_SHOW):
        image_array = sample_images[index]
        image = Image.fromarray(image_array)
        image.save(name+"_"+str(epoch).zfill(3) + "_" + str(index).zfill(3) +".png") 
  

In [ ]:
# Paths

OUTPUT_DIR = './gen-' + '{date:%Y-%m-%d_%H_%M_%S}/'.format(date=datetime.datetime.now())
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

SAMPLE_NOISE_FILE = OUTPUT_DIR + "sample_noise.csv"
noise_file_handle = open(SAMPLE_NOISE_FILE,'a')
    
for index in range(EPOCHS_TO_RUN):
    print('generating images, set ' + str(index))
    noise_data = np.random.normal(0, 1, size=(SAMPLES_TO_SHOW, NOISE_SIZE))
    predict(noise_data,index)
print("done")